## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Kavieng,PG,-2.5744,150.7967,81.64,76,3,9.04,clear sky,2021-08-01 20:55:54
1,1,Hermanus,ZA,-34.4187,19.2345,57.90,78,94,4.34,overcast clouds,2021-08-01 20:55:54
2,2,Upernavik,GL,72.7868,-56.1549,43.11,79,26,13.22,scattered clouds,2021-08-01 20:55:55
3,3,Vaini,TO,-21.2000,-175.2000,71.76,83,20,21.85,few clouds,2021-08-01 20:55:55
4,4,Zinder,NE,13.8072,8.9881,89.85,44,87,10.20,overcast clouds,2021-08-01 20:55:56


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Kavieng,PG,-2.5744,150.7967,81.64,76,3,9.04,clear sky,2021-08-01 20:55:54
4,4,Zinder,NE,13.8072,8.9881,89.85,44,87,10.20,overcast clouds,2021-08-01 20:55:56
5,5,Butaritari,KI,3.0707,172.7902,83.05,69,13,9.78,few clouds,2021-08-01 20:55:56
8,8,Albany,US,42.6001,-73.9662,76.87,74,82,1.01,broken clouds,2021-08-01 20:52:48
11,11,Saint-Georges,GD,12.0564,-61.7485,85.68,79,20,17.27,few clouds,2021-08-01 20:54:32
12,12,Alofi,NU,-19.0595,-169.9187,76.89,73,75,17.27,broken clouds,2021-08-01 20:55:58
13,13,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,68,20,13.80,few clouds,2021-08-01 20:55:58
15,15,Evanston,US,42.0411,-87.6901,79.72,62,40,5.99,scattered clouds,2021-08-01 20:51:43
21,21,Hamilton,US,39.1834,-84.5333,81.00,81,28,1.01,scattered clouds,2021-08-01 20:54:59
23,23,Hithadhoo,MV,-0.6000,73.0833,80.78,73,87,6.46,moderate rain,2021-08-01 20:56:00


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,81.64,clear sky,-2.5744,150.7967,
4,Zinder,NE,89.85,overcast clouds,13.8072,8.9881,
5,Butaritari,KI,83.05,few clouds,3.0707,172.7902,
8,Albany,US,76.87,broken clouds,42.6001,-73.9662,
11,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,
12,Alofi,NU,76.89,broken clouds,-19.0595,-169.9187,
13,Arraial Do Cabo,BR,75.16,few clouds,-22.9661,-42.0278,
15,Evanston,US,79.72,scattered clouds,42.0411,-87.6901,
21,Hamilton,US,81.00,scattered clouds,39.1834,-84.5333,
23,Hithadhoo,MV,80.78,moderate rain,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,81.64,clear sky,-2.5744,150.7967,Nusa Island Retreat
4,Zinder,NE,89.85,overcast clouds,13.8072,8.9881,Hotel de Ville
5,Butaritari,KI,83.05,few clouds,3.0707,172.7902,Isles Sunset Lodge
11,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
12,Alofi,NU,76.89,broken clouds,-19.0595,-169.9187,Taloa Heights


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and 
</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))